In [1]:
from dask.distributed import Client, LocalCluster
from dask import delayed
import dask
from wepredict.pytorch_regression import pytorch_linear
from wepredict.plink_reader import Genetic_data_read, get_genotypes
from wepredict.helpers import *

In [2]:
plink_file = '../data/chr10'
pheno_file = '../data/sim_1000G_chr10.txt'
ld_block_file = '../data/Berisa.EUR.hg19.bed'
data = Genetic_data_read(plink_file, ld_block_file, pheno_file)
train_index, valid_index, test_index = generate_valid_test_data(data.n, 0.1, 0.1)

In [19]:
cluster = LocalCluster(processes=False)
client = Client(cluster)

In [20]:
client

Client Scheduler: inproc://147.8.195.185/16140/1 Dashboard: http://localhost:32898/status,Cluster Workers: 1 Cores: 8 Memory: 16.61 GB


In [21]:
cluster.scale(2)

In [ ]:
out = list()
for ld_block in data.groups[10]:
    mat = delayed(get_genotypes)(22, ld_block, plink_file, data.sub_in)
    sample = delayed(get_samples)(mat, data.pheno['V1'].values,
                                  train_index, valid_index)
    del mat
    model = delayed(pytorch_linear)(sample['training_x'],
                                    sample['training_y'],
                                    sample['valid_x'],
                                    sample['valid_y'],
                                    mini_batch_size=100, type='c')
    results = delayed(model.run)('l1', lamb=0.01, epochs=100)
    out.append(results)

In [ ]:
res = dask.compute(out)

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://147.8.195.185/16140/10 remote=inproc://147.8.195.185/16140/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://147.8.195.185/16140/11 remote=inproc://147.8.195.185/16140/1>
